# Chapter 13: Sampling Plans

In [1]:
import numpy as np

## Algorithm 13.1

In [9]:
def samples_full_factorial(a, b, m):
    ranges = (np.linspace(a[i], b[i], m[i] ) for i in range(len(a)))
    return np.meshgrid(*ranges)

## Algorithm 13.2

In [19]:
def uniform_projection_plan(m, n):
    perms = [ np.random.permutation(m) for i in range(n) ]
    return [ [perms[i][j] for i in range(n)] for j in range(m) ]

## Algorithm 13.3

In [2]:
def pairwise_distances(X, p=2):
    m = len(X)
    return [np.linalg.norm(X[i]-X[j],p) for i in range(m-1) for j in range(i+1,m)]

### Example

In [5]:
X = np.array([[1.0, 3.0], [2.0, 4.1], [9.0,1.0]])
pairwise_distances(X)

[1.4866068747318502, 8.246211251235321, 7.6557168181692825]

## Algorithm 13.4

In [4]:
def compare_sampling_plans(A,B, p=2):
    pA = np.sort(pairwise_distances(A,p))
    pB = np.sort(pairwise_distances(B,p))
    for dA, dB in zip(pA, pB):
        if dA < dB:
            return 1
        elif dA > dB:
            return -1
    return 0

### Example

In [5]:
A = np.array([[1.0, 3.0], [2.0, 4.1], [9.0,1.0]])
B = np.array([[1.5, 2.1], [5.2, 9.5], [7.3, 2.0]])
compare_sampling_plans(A, B)

1

## Algorithm 13.5

In [14]:
def mutate(X):
    m, n = len(X), len(X[0])
    j = np.random.randint(n)
    i = np.random.permutation(m)[0:2]
    X[i[0]][j], X[i[1]][j] = X[i[1]][j], X[i[0]][j]
    return X

## Algorithm 13.6

In [8]:
def phiq(X, q=1, p=2):
    dists = np.array(pairwise_distances(X, p))
    return np.sum((dists**(-q))**(1/q))

## Algorithm 13.7

In [6]:
def min_dist(a, B, p):
    return np.min([np.linalg.norm(a-b, p) for b in B])
def d_max(A, B, p=2):
    return np.max([min_dist(a, B, p) for a in A]) 

### Example

In [4]:
A = np.array([[1.0, 3.0], [2.0, 4.1], [9.0,1.0]])
B = np.array([[1.5, 2.1], [5.2, 9.5], [7.3, 2.0]])
d_max(A,B)

2.06155281280883

## Algorithm 13.8

In [61]:
def greedy_local_search(X, m, d):
    S = [list(X[np.random.randint(0,m)])]
    for i in range(1,m):
        j = np.argmin([np.infty if list(x) in S else d(X, S + [list(x)]) for x in X])
        S.append(list(X[j]) )
    return S

### Example

In [63]:
X = np.array([[1.0, 3.0], [2.0, 4.1], [8.4,2.1], [1.9,3.0], [7.2,0.5], [3.3, 2.9]])
greedy_local_search(X,4,d_max)

[[8.4, 2.1], [2.0, 4.1], [7.2, 0.5], [1.9, 3.0]]

## Algorithm 13.9

In [26]:
def exchange_algorithm(X, m, d):
    ixs = list(np.random.permutation(m))
    S = [ list(X[ix]) for ix in ixs]
    delta, done = d(X, S), False
    while not done:
        best_pair = (0,0)
        for i in range(m):
            s = S[i]
            for j, x in enumerate(X):
                if list(x) not in S:
                    S[i] = list(x)
                    delta_prime = d(X,S)
                    if delta_prime < delta:
                        delta = delta_prime
                        best_pair = (i, j)
            S[i] = s
    
        done = best_pair == (0,0)
        if not done:
            i, j = best_pair
            S[i] = list(X[j])
    
    return S

### Example

In [27]:
X = np.array([[1.0, 3.0], [2.0, 4.1], [8.4,2.1], [1.9,3.0], [7.2,0.5], [3.3, 2.9]])
exchange_algorithm(X,4,d_max)

[[7.2, 0.5], [8.4, 2.1], [1.9, 3.0], [3.3, 2.9]]